In [1]:
# import requests
import pandas as pd

from airflow.models.dag import DAG
from airflow.operators.python import PythonOperator

from kraken_api import fetch_ohlc_data

from pandas_funcs import (
    convert_api_call_to_dataframe,
    select_columns
)

# from ge_funcs import ge_validation


ModuleNotFoundError: No module named 'pandas'

In [4]:
TICKER_PAIR = "XXBTZUSD"
COLUMN_NAMES = ["date", "open", "high", "low", "close", "vwap", "volume", "count"]
RELEVANT_COLUMNS = ["date", "open", "high", "low", "close"]

In [5]:
data = fetch_ohlc_data(TICKER_PAIR, "1d")

NameError: name 'fetch_ohlc_data' is not defined

In [11]:
df = convert_api_call_to_dataframe(data, COLUMN_NAMES)

In [12]:
df

,date,open,high,low,close,vwap,volume,count
0,2023-02-06,22938.2,23150.0,22599.9,22766.3,22887.0,3081.037288,28995
1,2023-02-07,22765.5,23347.7,22750.1,23253.2,23055.8,2505.208148,25138
2,2023-02-08,23253.3,23450.0,22675.5,22964.8,23049.1,2330.968641,30058
3,2023-02-09,22963.6,23008.2,21700.0,21792.0,22333.1,4427.297334,39050
4,2023-02-10,21792.0,21939.9,21475.0,21639.8,21716.1,2769.511937,27382
...,...,...,...,...,...,...,...,...
715,2025-01-21,102142.1,107200.0,100114.2,106162.8,104482.0,2766.178680,57011
716,2025-01-22,106162.8,106293.4,103271.3,103648.5,104664.0,1391.420726,36254
717,2025-01-23,103648.6,106833.2,101281.7,103920.2,104188.7,4692.190785,64479
718,2025-01-24,103920.2,107080.6,102800.0,104840.4,105315.2,1795.722536,36250


In [ ]:
df = select_columns(df, RELEVANT_COLUMNS)

In [14]:
df

,date,open,high,low,close
0,2023-02-06,22938.2,23150.0,22599.9,22766.3
1,2023-02-07,22765.5,23347.7,22750.1,23253.2
2,2023-02-08,23253.3,23450.0,22675.5,22964.8
3,2023-02-09,22963.6,23008.2,21700.0,21792.0
4,2023-02-10,21792.0,21939.9,21475.0,21639.8
...,...,...,...,...,...
715,2025-01-21,102142.1,107200.0,100114.2,106162.8
716,2025-01-22,106162.8,106293.4,103271.3,103648.5
717,2025-01-23,103648.6,106833.2,101281.7,103920.2
718,2025-01-24,103920.2,107080.6,102800.0,104840.4
